# Organizar os dados
 
Preparar os dados pré processados anteriormente para aplicação nas simulações da GAT



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mygene
import h5py
import networkx as nx
import seaborn as sns
%matplotlib inline


In [ ]:
# Vou utilizar sempre as features completas, ou seja, com o número máximo de genes que eu tenho disponível 
# Mesmo tendo features separadas para cada uma das redes pelo trabalho do Shulte-Sasse o numero muitas vezes não bate exatamente
# Provavelmente mesmo com um maior número de genes também não vai bater 100% mas a chance é maior

# O motivo é que o número de genes contidos nos nós de uma rede pode ser váriavel mas o número máximo de genes com features que eu tenho disponível é 19.666
# Logo esse número sempre vai encaixar nas redes, cobrindo o maior número possivel de genes contidos nessa rede

# A organização nesse notebook é para preparar os dados para a biblioteca do stellargraph
# Adicionar os pesos 1 para as redes PPI (renomear colunas para source e target)
# Setar o gene como index do dataframe de features 
# Juntar as features com as centralidades das redes
# Setar o gene como index no dataframe de labels

### Multinet

In [33]:
MULTINET_network = pd.read_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/networks/MULTINET_network3.tsv', sep='\t', index_col='Unnamed: 0')

MULTINET_features_da_rede = pd.read_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/features_das_redes/features_MultiNet.tsv', sep='\t', index_col='Unnamed: 0')

features = pd.read_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/features/unity_features.tsv', sep='\t', index_col='Unnamed: 0')

labels = pd.read_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/labels/unity_labels.tsv', sep='\t', index_col='Unnamed: 0')


In [34]:
MULTINET_network.insert(len(MULTINET_network.columns), 'weight', 1)

MULTINET_network2 = MULTINET_network.rename(columns={"gene1": "source", "gene2": "target"})

MULTINET_network2

,source,target,weight
0,RNF14,VDR,1
1,ACVR1,RNF14,1
2,DYRK2,RNF14,1
3,RNF14,UBE2E1,1
4,RNF14,UBE2D4,1
...,...,...,...
108599,LDHB,LDHD,1
108600,LDHC,LDHD,1
108601,LDHA,LDHB,1
108602,LDHA,LDHC,1


In [11]:
MULTINET_network2.to_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/data_final/MULTINET_network.tsv', sep='\t')

In [37]:
MULTINET_features_da_rede2 = MULTINET_features_da_rede.rename(columns={"Gene": "gene"})

# Deixando igual o número de features ao número de nós 

# A ideia é fazer um conact com as features das redes (que já são os números de nós das redes) e manter só os que deram macth com esse dataframe 'left' no caso
# Isso soluciona porque concatena e fica apenas os genes contidos na rede ppi correspondente

MULTINET_features2 = pd.merge(MULTINET_features_da_rede2, features, how = 'left', on = 'gene')

MULTINET_features_completo = MULTINET_features2.set_index('gene')

MULTINET_features_completo

,betweenness,closeness,degree,clustering_coefficient,CNA: BLCA,CNA: BRCA,CNA: CESC,CNA: COAD,CNA: ESCA,CNA: HNSC,...,MF: KIRC,MF: KIRP,MF: LIHC,MF: LUAD,MF: LUSC,MF: PRAD,MF: READ,MF: STAD,MF: THCA,MF: UCEC
gene,,,,,,,,,,,,,,,,,,,,,
A1BG,0.000000e+00,1.000000,0.000000,0.000000,0.0,0.000000,0.167401,0.000000,0.000000,0.255263,...,0.012838,0.040970,0.050114,0.000000,0.000000,0.022774,0.028288,0.013054,0.000000,0.007712
A1CF,1.253962e-03,0.000008,0.002679,0.100000,0.0,0.000000,0.000000,0.000000,0.283951,0.000000,...,0.012698,0.000000,0.024672,0.117901,0.000000,0.090341,0.027762,0.011448,0.000000,0.010251
A2M,2.633543e-03,0.000011,0.012726,0.036842,0.0,0.000000,0.000000,0.174419,0.000000,0.000000,...,0.012836,0.040877,0.000000,0.122328,0.109769,0.022773,0.000000,0.038998,0.003601,0.019196
AAAS,6.191113e-05,0.000010,0.000670,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.025661,0.040872,0.000000,0.000000,0.000000,0.000000,0.000000,0.012746,0.003604,0.003829
AACS,1.092119e-05,0.000008,0.008707,0.670330,0.0,0.227126,0.000000,0.000000,0.000000,0.159091,...,0.000000,0.000000,0.000000,0.020504,0.000000,0.000000,0.028397,0.038616,0.003602,0.011557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZXDC,5.593014e-07,0.000008,0.001340,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.040907,0.000000,0.000000,0.000000,0.000000,0.000000,0.025721,0.003603,0.011466
ZYG11B,0.000000e+00,0.000007,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.012818,0.081662,0.025013,0.000000,0.000000,0.000000,0.000000,0.038673,0.000000,0.000000
ZYX,5.255937e-03,0.000014,0.097790,0.063554,0.0,0.000000,0.264317,0.000000,0.000000,0.000000,...,0.012815,0.000000,0.000000,0.020495,0.000000,0.000000,0.000000,0.012873,0.000000,0.022981


In [28]:
MULTINET_features_completo.to_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/data_final/MULTINET_features.tsv', sep='\t')

In [39]:
# Ficar apenas com os labels que também contém no banco de dados

label_1 = pd.merge(MULTINET_features_da_rede2, labels, how = 'left', on = 'gene')

label_2 = label_1.drop(columns=['betweenness', 'closeness', 'degree', 'clustering_coefficient'])

# label_2.loc[label_2['label'] == 'True', 'label'] = 'driver'
# label_2.loc[label_2['label'] == 'False', 'label'] = 'passenger'

label_3 = label_2.set_index('gene')

label_3

,label
gene,
A1BG,False
A1CF,True
A2M,False
AAAS,False
AACS,False
...,...
ZXDC,False
ZYG11B,False
ZYX,False


In [42]:
label_3.isnull().sum().sum()

0

In [13]:
label_3.to_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/data_final/MULTINET_labels.tsv', sep='\t')

### IRefIndex

In [50]:
IREF_network = pd.read_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/networks/IREF_network3.tsv', sep='\t', index_col='Unnamed: 0')

IREF_features_da_rede = pd.read_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/features_das_redes/features_IRefIndex.tsv', sep='\t', index_col='Unnamed: 0')

features = pd.read_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/features/unity_features.tsv', sep='\t', index_col='Unnamed: 0')

labels = pd.read_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/labels/unity_labels.tsv', sep='\t', index_col='Unnamed: 0')


In [51]:
IREF_network.insert(len(IREF_network.columns), 'weight', 1)

IREF_network2 = IREF_network.rename(columns={"gene1": "source", "gene2": "target"})

IREF_network2

,source,target,weight
0,AR,RNF14,1
1,RNF14,TCF3,1
2,RNF14,UBE2E1,1
3,RNF14,UBE2D4,1
4,RNF14,UBE2D3,1
...,...,...,...
133129,SELL,VCAN,1
133130,SELP,VCAN,1
133131,CAND1,LDHD,1
133132,CAND1,LDHA,1


In [52]:
# Criar lista de NaN para removover

IREF_features_da_rede2 = IREF_features_da_rede.rename(columns={"Gene": "gene"})

label_1 = pd.merge(IREF_features_da_rede2, labels, how = 'left', on = 'gene')

label_2 = label_1.drop(columns=['betweenness', 'closeness', 'degree', 'clustering_coefficient'])

nan_values = label_2[label_2['label'].isna()]

nan_values

,gene,label
4921,GOLGA8K,NaN
9621,PRAMEF14,NaN
10067,RAB6D,NaN
12257,SYNJ2BP-COX16,NaN
13152,TRIM64C,NaN


In [53]:
# Traduzir na rede PPI os genes para NaN e remover todas as aparições

IREF_network2.replace(nan_values.set_index('gene').to_dict()['label'], inplace = True)

IREF_network3 = IREF_network2.dropna()

IREF_network3

,source,target,weight
0,AR,RNF14,1
1,RNF14,TCF3,1
2,RNF14,UBE2E1,1
3,RNF14,UBE2D4,1
4,RNF14,UBE2D3,1
...,...,...,...
133129,SELL,VCAN,1
133130,SELP,VCAN,1
133131,CAND1,LDHD,1
133132,CAND1,LDHA,1


In [54]:
IREF_network3.to_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/data_final/IREF_network.tsv', sep='\t')

In [55]:
IREF_features_da_rede2 = IREF_features_da_rede.rename(columns={"Gene": "gene"})

# Deixando igual o número de features ao número de nós 

# A ideia é fazer um conact com as features das redes (que já são os números de nós das redes) e manter só os que deram macth com esse dataframe 'left' no caso
# Isso soluciona porque concatena e fica apenas os genes contidos na rede ppi correspondente

IREF_features2 = pd.merge(IREF_features_da_rede2, features, how = 'left', on = 'gene')

IREF_features_completo = IREF_features2.set_index('gene')

IREF_features_completo

,betweenness,closeness,degree,clustering_coefficient,CNA: BLCA,CNA: BRCA,CNA: CESC,CNA: COAD,CNA: ESCA,CNA: HNSC,...,MF: KIRC,MF: KIRP,MF: LIHC,MF: LUAD,MF: LUSC,MF: PRAD,MF: READ,MF: STAD,MF: THCA,MF: UCEC
gene,,,,,,,,,,,,,,,,,,,,,
A1BG,0.000209,0.000011,0.001145,0.022222,0.0,0.000000,0.167401,0.000000,0.000000,0.255263,...,0.012838,0.040970,0.050114,0.000000,0.000000,0.022774,0.028288,0.013054,0.000000,0.007712
A1CF,0.000009,0.000010,0.000891,0.035714,0.0,0.000000,0.000000,0.000000,0.283951,0.000000,...,0.012698,0.000000,0.024672,0.117901,0.000000,0.090341,0.027762,0.011448,0.000000,0.010251
A2M,0.001120,0.000013,0.011962,0.034938,0.0,0.000000,0.000000,0.174419,0.000000,0.000000,...,0.012836,0.040877,0.000000,0.122328,0.109769,0.022773,0.000000,0.038998,0.003601,0.019196
AAAS,0.000002,0.000010,0.000255,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.025661,0.040872,0.000000,0.000000,0.000000,0.000000,0.000000,0.012746,0.003604,0.003829
AACS,0.000003,0.000016,0.000127,0.000000,0.0,0.227126,0.000000,0.000000,0.000000,0.159091,...,0.000000,0.000000,0.000000,0.020504,0.000000,0.000000,0.028397,0.038616,0.003602,0.011557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZYG11A,0.000000,0.000016,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.007645
ZYG11B,0.000008,0.000016,0.000764,0.571429,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.012818,0.081662,0.025013,0.000000,0.000000,0.000000,0.000000,0.038673,0.000000,0.000000
ZYX,0.000580,0.000017,0.008781,0.081573,0.0,0.000000,0.264317,0.000000,0.000000,0.000000,...,0.012815,0.000000,0.000000,0.020495,0.000000,0.000000,0.000000,0.012873,0.000000,0.022981


In [56]:
IREF_features_completo2 = IREF_features_completo.dropna()

IREF_features_completo2

,betweenness,closeness,degree,clustering_coefficient,CNA: BLCA,CNA: BRCA,CNA: CESC,CNA: COAD,CNA: ESCA,CNA: HNSC,...,MF: KIRC,MF: KIRP,MF: LIHC,MF: LUAD,MF: LUSC,MF: PRAD,MF: READ,MF: STAD,MF: THCA,MF: UCEC
gene,,,,,,,,,,,,,,,,,,,,,
A1BG,0.000209,0.000011,0.001145,0.022222,0.0,0.000000,0.167401,0.000000,0.000000,0.255263,...,0.012838,0.040970,0.050114,0.000000,0.000000,0.022774,0.028288,0.013054,0.000000,0.007712
A1CF,0.000009,0.000010,0.000891,0.035714,0.0,0.000000,0.000000,0.000000,0.283951,0.000000,...,0.012698,0.000000,0.024672,0.117901,0.000000,0.090341,0.027762,0.011448,0.000000,0.010251
A2M,0.001120,0.000013,0.011962,0.034938,0.0,0.000000,0.000000,0.174419,0.000000,0.000000,...,0.012836,0.040877,0.000000,0.122328,0.109769,0.022773,0.000000,0.038998,0.003601,0.019196
AAAS,0.000002,0.000010,0.000255,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.025661,0.040872,0.000000,0.000000,0.000000,0.000000,0.000000,0.012746,0.003604,0.003829
AACS,0.000003,0.000016,0.000127,0.000000,0.0,0.227126,0.000000,0.000000,0.000000,0.159091,...,0.000000,0.000000,0.000000,0.020504,0.000000,0.000000,0.028397,0.038616,0.003602,0.011557
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZYG11A,0.000000,0.000016,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.007645
ZYG11B,0.000008,0.000016,0.000764,0.571429,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.012818,0.081662,0.025013,0.000000,0.000000,0.000000,0.000000,0.038673,0.000000,0.000000
ZYX,0.000580,0.000017,0.008781,0.081573,0.0,0.000000,0.264317,0.000000,0.000000,0.000000,...,0.012815,0.000000,0.000000,0.020495,0.000000,0.000000,0.000000,0.012873,0.000000,0.022981


In [57]:
IREF_features_completo2.to_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/data_final/IREF_features.tsv', sep='\t')

In [58]:
# Ficar apenas com os labels que também contém no banco de dados

label_1 = pd.merge(IREF_features_da_rede2, labels, how = 'left', on = 'gene')

label_2 = label_1.drop(columns=['betweenness', 'closeness', 'degree', 'clustering_coefficient'])

# label_2.loc[label_2['label'] == 'True', 'label'] = 'driver'
# label_2.loc[label_2['label'] == 'False', 'label'] = 'passenger'

label_3 = label_2.set_index('gene')

label_3

,label
gene,
A1BG,False
A1CF,True
A2M,False
AAAS,False
AACS,False
...,...
ZYG11A,False
ZYG11B,False
ZYX,False


In [47]:
label_3.isnull().sum().sum()

5

In [59]:
label_4 = label_3.dropna()

label_4

,label
gene,
A1BG,False
A1CF,True
A2M,False
AAAS,False
AACS,False
...,...
ZYG11A,False
ZYG11B,False
ZYX,False


In [60]:
label_4.to_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/data_final/IREF_labels.tsv', sep='\t')

### HPRD

In [2]:
HPRD_network = pd.read_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/networks/HPRD_network3.tsv', sep='\t', index_col='Unnamed: 0')

HPRD_features_da_rede = pd.read_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/features_das_redes/features_HPRD.tsv', sep='\t', index_col='Unnamed: 0')

features = pd.read_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/features/unity_features.tsv', sep='\t', index_col='Unnamed: 0')

labels = pd.read_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/labels/unity_labels.tsv', sep='\t', index_col='Unnamed: 0')


In [3]:
HPRD_network.insert(len(HPRD_network.columns), 'weight', 1)

HPRD_network2 = HPRD_network.rename(columns={"gene1": "source", "gene2": "target"})

HPRD_network2

,source,target,weight
0,NR3C1,RNF14,1
1,NCOA4,RNF14,1
2,ESR1,RNF14,1
3,RNF14,UBE2E1,1
4,RNF14,TENT5A,1
...,...,...,...
36859,SELE,SELL,1
36860,CRYAA,CRYAB,1
36861,PLAT,PLAU,1
36862,NFIB,NFIC,1


In [33]:
HPRD_network2.to_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/data_final/HPRD_network.tsv', sep='\t')

In [4]:
HPRD_features_da_rede2 = HPRD_features_da_rede.rename(columns={"Gene": "gene"})

# Deixando igual o número de features ao número de nós 

# A ideia é fazer um conact com as features das redes (que já são os números de nós das redes) e manter só os que deram macth com esse dataframe 'left' no caso
# Isso soluciona porque concatena e fica apenas os genes contidos na rede ppi correspondente

HPRD_features2 = pd.merge(HPRD_features_da_rede2, features, how = 'left', on = 'gene')

HPRD_features_completo = HPRD_features2.set_index('gene')

HPRD_features_completo

,betweenness,closeness,degree,clustering_coefficient,CNA: BLCA,CNA: BRCA,CNA: CESC,CNA: COAD,CNA: ESCA,CNA: HNSC,...,MF: KIRC,MF: KIRP,MF: LIHC,MF: LUAD,MF: LUSC,MF: PRAD,MF: READ,MF: STAD,MF: THCA,MF: UCEC
gene,,,,,,,,,,,,,,,,,,,,,
A1BG,0.000000,1.000000,0.000000,0.000000,0.0,0.0,0.167401,0.000000,0.000000,0.255263,...,0.012838,0.040970,0.050114,0.000000,0.000000,0.022774,0.028288,0.013054,0.000000,0.007712
A1CF,0.000529,0.000012,0.014925,0.100000,0.0,0.0,0.000000,0.000000,0.283951,0.000000,...,0.012698,0.000000,0.024672,0.117901,0.000000,0.090341,0.027762,0.011448,0.000000,0.010251
A2M,0.045448,0.000018,0.100746,0.010582,0.0,0.0,0.000000,0.174419,0.000000,0.000000,...,0.012836,0.040877,0.000000,0.122328,0.109769,0.022773,0.000000,0.038998,0.003601,0.019196
AAGAB,0.000095,0.000014,0.003731,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.012804,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
AAMP,0.000000,0.000012,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.396465,...,0.000000,0.040897,0.024970,0.000000,0.000000,0.000000,0.027894,0.000000,0.000000,0.003815
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZWINT,0.000784,0.000012,0.011194,0.333333,0.0,0.0,0.000000,0.000000,0.283951,0.000000,...,0.000000,0.040867,0.000000,0.061462,0.000000,0.022759,0.027911,0.026028,0.000000,0.011423
ZXDC,0.000023,0.000014,0.007463,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.040907,0.000000,0.000000,0.000000,0.000000,0.000000,0.025721,0.003603,0.011466
ZYG11B,0.000000,0.000011,0.003731,1.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.012818,0.081662,0.025013,0.000000,0.000000,0.000000,0.000000,0.038673,0.000000,0.000000


In [26]:
HPRD_features_completo2 = HPRD_features_completo.dropna()

In [27]:
HPRD_features_completo2

,betweenness,closeness,degree,clustering_coefficient,CNA: BLCA,CNA: BRCA,CNA: CESC,CNA: COAD,CNA: ESCA,CNA: HNSC,...,MF: KIRC,MF: KIRP,MF: LIHC,MF: LUAD,MF: LUSC,MF: PRAD,MF: READ,MF: STAD,MF: THCA,MF: UCEC
gene,,,,,,,,,,,,,,,,,,,,,
A1BG,0.000000,1.000000,0.000000,0.000000,0.0,0.0,0.167401,0.000000,0.000000,0.255263,...,0.012838,0.040970,0.050114,0.000000,0.000000,0.022774,0.028288,0.013054,0.000000,0.007712
A1CF,0.000529,0.000012,0.014925,0.100000,0.0,0.0,0.000000,0.000000,0.283951,0.000000,...,0.012698,0.000000,0.024672,0.117901,0.000000,0.090341,0.027762,0.011448,0.000000,0.010251
A2M,0.045448,0.000018,0.100746,0.010582,0.0,0.0,0.000000,0.174419,0.000000,0.000000,...,0.012836,0.040877,0.000000,0.122328,0.109769,0.022773,0.000000,0.038998,0.003601,0.019196
AAGAB,0.000095,0.000014,0.003731,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.012804,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
AAMP,0.000000,0.000012,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.396465,...,0.000000,0.040897,0.024970,0.000000,0.000000,0.000000,0.027894,0.000000,0.000000,0.003815
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZWINT,0.000784,0.000012,0.011194,0.333333,0.0,0.0,0.000000,0.000000,0.283951,0.000000,...,0.000000,0.040867,0.000000,0.061462,0.000000,0.022759,0.027911,0.026028,0.000000,0.011423
ZXDC,0.000023,0.000014,0.007463,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.040907,0.000000,0.000000,0.000000,0.000000,0.000000,0.025721,0.003603,0.011466
ZYG11B,0.000000,0.000011,0.003731,1.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.012818,0.081662,0.025013,0.000000,0.000000,0.000000,0.000000,0.038673,0.000000,0.000000


In [28]:
HPRD_features_completo2.to_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/data_final/HPRD_features.tsv', sep='\t')

In [13]:
# Ficar apenas com os labels que também contém no banco de dados

label_1 = pd.merge(HPRD_features_da_rede2, labels, how = 'left', on = 'gene')

label_2 = label_1.drop(columns=['betweenness', 'closeness', 'degree', 'clustering_coefficient'])

# label_2.loc[label_2['label'] == 'True', 'label'] = 'driver'
# label_2.loc[label_2['label'] == 'False', 'label'] = 'passenger'

# label_3 = label_2.set_index('gene')

# label_3

In [14]:
nan_values = label_2[label_2['label'].isna()]

nan_values

,gene,label
122,ADAM3B,NaN
987,CACTIN-AS1,NaN
5125,MYCLP1,NaN
8489,TMSB4XP4,NaN
8643,TRD,NaN
8652,TRG,NaN


In [16]:
# Tentativa 1: tarduzir a lista de genes dentro da rede e substituir para NaN e depois remover

HPRD_network2.replace(nan_values.set_index('gene').to_dict()['label'], inplace = True)



In [17]:
HPRD_network2


,source,target,weight
0,NR3C1,RNF14,1
1,NCOA4,RNF14,1
2,ESR1,RNF14,1
3,RNF14,UBE2E1,1
4,RNF14,TENT5A,1
...,...,...,...
36859,SELE,SELL,1
36860,CRYAA,CRYAB,1
36861,PLAT,PLAU,1
36862,NFIB,NFIC,1


In [23]:
HPRD_network3 = HPRD_network2.dropna()

In [24]:
HPRD_network3.isnull().sum().sum()

0

In [25]:
HPRD_network3

,source,target,weight
0,NR3C1,RNF14,1
1,NCOA4,RNF14,1
2,ESR1,RNF14,1
3,RNF14,UBE2E1,1
4,RNF14,TENT5A,1
...,...,...,...
36859,SELE,SELL,1
36860,CRYAA,CRYAB,1
36861,PLAT,PLAU,1
36862,NFIB,NFIC,1


In [29]:
HPRD_network3.to_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/data_final/HPRD_network.tsv', sep='\t')

In [ ]:
# Com essa solução eu tenho a lista de genes dessa rede que não possuem features correspondentes
# O próximo passo é remover esses genes da rede

In [30]:
label_1 = pd.merge(HPRD_features_da_rede2, labels, how = 'left', on = 'gene')

label_2 = label_1.drop(columns=['betweenness', 'closeness', 'degree', 'clustering_coefficient'])

# label_2.loc[label_2['label'] == 'True', 'label'] = 'driver'
# label_2.loc[label_2['label'] == 'False', 'label'] = 'passenger'

label_3 = label_2.set_index('gene')

label_3

,label
gene,
A1BG,False
A1CF,True
A2M,False
AAGAB,False
AAMP,False
...,...
ZWINT,False
ZXDC,False
ZYG11B,False


In [31]:
label_4 = label_3.dropna()

label_4

,label
gene,
A1BG,False
A1CF,True
A2M,False
AAGAB,False
AAMP,False
...,...
ZWINT,False
ZXDC,False
ZYG11B,False


In [32]:
label_4.to_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/data_final/HPRD_labels.tsv', sep='\t')

### CPDB

In [61]:
CPDB_network = pd.read_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/networks/CPDB_network3.tsv', sep='\t', index_col='Unnamed: 0')

CPDB_features_da_rede = pd.read_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/features_das_redes/features_cpdb.tsv', sep='\t', index_col='Unnamed: 0')

features = pd.read_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/features/unity_features.tsv', sep='\t', index_col='Unnamed: 0')

labels = pd.read_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/labels/unity_labels.tsv', sep='\t', index_col='Unnamed: 0')


C:\Users\renan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [62]:
CPDB_network.insert(len(CPDB_network.columns), 'weight', 1)

CPDB_network2 = CPDB_network.rename(columns={"gene1": "source", "gene2": "target"})

CPDB_network2

,source,target,weight
0,RNF14,VDR,1
1,RNF14,SMAD4,1
2,RNF14,UBE2D4,1
3,RNF14,UBE2D2,1
4,RNF14,UBE2D3,1
...,...,...,...
1644080,CFL1,SERPINH1,1
1644081,DNM1L,PLEKHG1,1
1644082,MRPL23,SERPINH1,1
1644083,NFIA,NFIX,1


In [63]:
# Criar lista de NaN para removover

CPDB_features_da_rede2 = CPDB_features_da_rede.rename(columns={"Gene": "gene"})

label_1 = pd.merge(CPDB_features_da_rede2, labels, how = 'left', on = 'gene')

label_2 = label_1.drop(columns=['betweenness', 'closeness', 'degree', 'clustering_coefficient'])

nan_values = label_2[label_2['label'].isna()]

nan_values

,gene,label
1557,C16orf82,NaN
8323,MMP24OS,NaN
14601,TRAV12-2,NaN
14602,TRAV12-3,NaN
14603,TRAV21,NaN
14605,TRBV28,NaN
14606,TRBV5-1,NaN
14607,TRBV6-5,NaN


In [64]:
# Traduzir na rede PPI os genes para NaN e remover todas as aparições

CPDB_network2.replace(nan_values.set_index('gene').to_dict()['label'], inplace = True)

CPDB_network3 = CPDB_network2.dropna()

CPDB_network3

,source,target,weight
0,RNF14,VDR,1
1,RNF14,SMAD4,1
2,RNF14,UBE2D4,1
3,RNF14,UBE2D2,1
4,RNF14,UBE2D3,1
...,...,...,...
1644080,CFL1,SERPINH1,1
1644081,DNM1L,PLEKHG1,1
1644082,MRPL23,SERPINH1,1
1644083,NFIA,NFIX,1


In [65]:
CPDB_network3.to_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/data_final/CPDB_network.tsv', sep='\t')

In [66]:
CPDB_features_da_rede2 = CPDB_features_da_rede.rename(columns={"Gene": "gene"})

# Deixando igual o número de features ao número de nós 

# A ideia é fazer um conact com as features das redes (que já são os números de nós das redes) e manter só os que deram macth com esse dataframe 'left' no caso
# Isso soluciona porque concatena e fica apenas os genes contidos na rede ppi correspondente

CPDB_features2 = pd.merge(CPDB_features_da_rede2, features, how = 'left', on = 'gene')

CPDB_features_completo = CPDB_features2.set_index('gene')

CPDB_features_completo

,betweenness,closeness,degree,clustering_coefficient,CNA: BLCA,CNA: BRCA,CNA: CESC,CNA: COAD,CNA: ESCA,CNA: HNSC,...,MF: KIRC,MF: KIRP,MF: LIHC,MF: LUAD,MF: LUSC,MF: PRAD,MF: READ,MF: STAD,MF: THCA,MF: UCEC
gene,,,,,,,,,,,,,,,,,,,,,
A1BG,0.002217,0.000009,0.004098,0.045455,0.0,0.0,0.167401,0.000000,0.000000,0.255263,...,0.012838,0.040970,0.050114,0.000000,0.000000,0.022774,0.028288,0.013054,0.000000,0.007712
A1CF,0.000063,0.000010,0.004471,0.089744,0.0,0.0,0.000000,0.000000,0.283951,0.000000,...,0.012698,0.000000,0.024672,0.117901,0.000000,0.090341,0.027762,0.011448,0.000000,0.010251
A2M,0.015079,0.000014,0.085693,0.159081,0.0,0.0,0.000000,0.174419,0.000000,0.000000,...,0.012836,0.040877,0.000000,0.122328,0.109769,0.022773,0.000000,0.038998,0.003601,0.019196
A2ML1,0.000759,0.000014,0.076379,0.711295,0.0,0.0,0.000000,0.184049,0.000000,0.000000,...,0.000000,0.040866,0.049805,0.040017,0.054533,0.022713,0.000000,0.063755,0.000000,0.007600
A4GALT,0.000005,0.000006,0.001490,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.022728,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZXDC,0.001567,0.000012,0.032042,0.739642,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.040907,0.000000,0.000000,0.000000,0.000000,0.000000,0.025721,0.003603,0.011466
ZYG11B,0.000371,0.000012,0.026826,0.554033,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.012818,0.081662,0.025013,0.000000,0.000000,0.000000,0.000000,0.038673,0.000000,0.000000
ZYX,0.008109,0.000017,0.476155,0.720111,0.0,0.0,0.264317,0.000000,0.000000,0.000000,...,0.012815,0.000000,0.000000,0.020495,0.000000,0.000000,0.000000,0.012873,0.000000,0.022981


In [67]:
CPDB_features_completo2 = CPDB_features_completo.dropna()

CPDB_features_completo2

,betweenness,closeness,degree,clustering_coefficient,CNA: BLCA,CNA: BRCA,CNA: CESC,CNA: COAD,CNA: ESCA,CNA: HNSC,...,MF: KIRC,MF: KIRP,MF: LIHC,MF: LUAD,MF: LUSC,MF: PRAD,MF: READ,MF: STAD,MF: THCA,MF: UCEC
gene,,,,,,,,,,,,,,,,,,,,,
A1BG,0.002217,0.000009,0.004098,0.045455,0.0,0.0,0.167401,0.000000,0.000000,0.255263,...,0.012838,0.040970,0.050114,0.000000,0.000000,0.022774,0.028288,0.013054,0.000000,0.007712
A1CF,0.000063,0.000010,0.004471,0.089744,0.0,0.0,0.000000,0.000000,0.283951,0.000000,...,0.012698,0.000000,0.024672,0.117901,0.000000,0.090341,0.027762,0.011448,0.000000,0.010251
A2M,0.015079,0.000014,0.085693,0.159081,0.0,0.0,0.000000,0.174419,0.000000,0.000000,...,0.012836,0.040877,0.000000,0.122328,0.109769,0.022773,0.000000,0.038998,0.003601,0.019196
A2ML1,0.000759,0.000014,0.076379,0.711295,0.0,0.0,0.000000,0.184049,0.000000,0.000000,...,0.000000,0.040866,0.049805,0.040017,0.054533,0.022713,0.000000,0.063755,0.000000,0.007600
A4GALT,0.000005,0.000006,0.001490,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.022728,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZXDC,0.001567,0.000012,0.032042,0.739642,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.040907,0.000000,0.000000,0.000000,0.000000,0.000000,0.025721,0.003603,0.011466
ZYG11B,0.000371,0.000012,0.026826,0.554033,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.012818,0.081662,0.025013,0.000000,0.000000,0.000000,0.000000,0.038673,0.000000,0.000000
ZYX,0.008109,0.000017,0.476155,0.720111,0.0,0.0,0.264317,0.000000,0.000000,0.000000,...,0.012815,0.000000,0.000000,0.020495,0.000000,0.000000,0.000000,0.012873,0.000000,0.022981


In [68]:
CPDB_features_completo2.to_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/data_final/CPDB_features.tsv', sep='\t')

In [69]:
# Ficar apenas com os labels que também contém no banco de dados

label_1 = pd.merge(CPDB_features_da_rede2, labels, how = 'left', on = 'gene')

label_2 = label_1.drop(columns=['betweenness', 'closeness', 'degree', 'clustering_coefficient'])

# label_2.loc[label_2['label'] == 'True', 'label'] = 'driver'
# label_2.loc[label_2['label'] == 'False', 'label'] = 'passenger'

label_3 = label_2.set_index('gene')

label_3

,label
gene,
A1BG,False
A1CF,True
A2M,False
A2ML1,False
A4GALT,False
...,...
ZXDC,False
ZYG11B,False
ZYX,False


In [70]:
label_4 = label_3.dropna()

label_4

,label
gene,
A1BG,False
A1CF,True
A2M,False
A2ML1,False
A4GALT,False
...,...
ZXDC,False
ZYG11B,False
ZYX,False


In [71]:
label_4.to_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/data_final/CPDB_labels.tsv', sep='\t')

### STRING

In [72]:
STRING_network = pd.read_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/networks/STRING_network3.tsv', sep='\t', index_col='Unnamed: 0')

STRING_features_da_rede = pd.read_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/features_das_redes/features_STRING.tsv', sep='\t', index_col='Unnamed: 0')

features = pd.read_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/features/unity_features.tsv', sep='\t', index_col='Unnamed: 0')

labels = pd.read_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/labels/unity_labels.tsv', sep='\t', index_col='Unnamed: 0')


C:\Users\renan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [73]:
STRING_network.insert(len(STRING_network.columns), 'weight', 1)

STRING_network2 = STRING_network.rename(columns={"gene1": "source", "gene2": "target"})

STRING_network2

,source,target,weight
0,RNF14,UBE2Q1,1
1,RNF14,UBE2Q2,1
2,RNF11,RNF14,1
3,PPP3R1,RNF14,1
4,HSPA4,RNF14,1
...,...,...,...
5031940,SLC7A13,SLC7A14,1
5031941,SELE,SELP,1
5031942,SLC7A10,SLC7A13,1
5031943,SLC7A11,SLC7A13,1


In [74]:
# Criar lista de NaN para removover

STRING_features_da_rede2 = STRING_features_da_rede.rename(columns={"Gene": "gene"})

label_1 = pd.merge(STRING_features_da_rede2, labels, how = 'left', on = 'gene')

label_2 = label_1.drop(columns=['betweenness', 'closeness', 'degree', 'clustering_coefficient'])

nan_values = label_2[label_2['label'].isna()]

nan_values

,gene,label
150,ACOD1,NaN
399,AGAP6,NaN
472,AKAIN1,NaN
1010,ARMH2,NaN
1496,BLACE,NaN
...,...,...
17215,XAGE5,NaN
17824,ZNF625-ZNF20,NaN
17878,ZNF705B,NaN
17930,ZNF788P,NaN


In [75]:
# Traduzir na rede PPI os genes para NaN e remover todas as aparições

STRING_network2.replace(nan_values.set_index('gene').to_dict()['label'], inplace = True)

STRING_network3 = STRING_network2.dropna()

STRING_network3

,source,target,weight
0,RNF14,UBE2Q1,1
1,RNF14,UBE2Q2,1
2,RNF11,RNF14,1
3,PPP3R1,RNF14,1
4,HSPA4,RNF14,1
...,...,...,...
5031940,SLC7A13,SLC7A14,1
5031941,SELE,SELP,1
5031942,SLC7A10,SLC7A13,1
5031943,SLC7A11,SLC7A13,1


In [76]:
STRING_network3.to_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/data_final/STRING_network.tsv', sep='\t')

In [77]:
STRING_features_da_rede2 = STRING_features_da_rede.rename(columns={"Gene": "gene"})

# Deixando igual o número de features ao número de nós 

# A ideia é fazer um conact com as features das redes (que já são os números de nós das redes) e manter só os que deram macth com esse dataframe 'left' no caso
# Isso soluciona porque concatena e fica apenas os genes contidos na rede ppi correspondente

STRING_features2 = pd.merge(STRING_features_da_rede2, features, how = 'left', on = 'gene')

STRING_features_completo = STRING_features2.set_index('gene')

STRING_features_completo

,betweenness,closeness,degree,clustering_coefficient,CNA: BLCA,CNA: BRCA,CNA: CESC,CNA: COAD,CNA: ESCA,CNA: HNSC,...,MF: KIRC,MF: KIRP,MF: LIHC,MF: LUAD,MF: LUSC,MF: PRAD,MF: READ,MF: STAD,MF: THCA,MF: UCEC
gene,,,,,,,,,,,,,,,,,,,,,
A1BG,0.002394,0.574926,0.056669,0.374335,0.0,0.0,0.167401,0.000000,0.000000,0.255263,...,0.012838,0.040970,0.050114,0.000000,0.000000,0.022774,0.028288,0.013054,0.000000,0.007712
A1CF,0.000439,0.491816,0.016191,0.216980,0.0,0.0,0.000000,0.000000,0.283951,0.000000,...,0.012698,0.000000,0.024672,0.117901,0.000000,0.090341,0.027762,0.011448,0.000000,0.010251
A2M,0.003584,0.595557,0.071267,0.274020,0.0,0.0,0.000000,0.174419,0.000000,0.000000,...,0.012836,0.040877,0.000000,0.122328,0.109769,0.022773,0.000000,0.038998,0.003601,0.019196
A2ML1,0.001648,0.451462,0.018447,0.170606,0.0,0.0,0.000000,0.184049,0.000000,0.000000,...,0.000000,0.040866,0.049805,0.040017,0.054533,0.022713,0.000000,0.063755,0.000000,0.007600
A4GALT,0.000348,0.482407,0.009821,0.179099,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.022728,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZYG11A,0.000080,0.354566,0.006370,0.380102,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.007645
ZYG11B,0.000280,0.406438,0.008626,0.265268,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.012818,0.081662,0.025013,0.000000,0.000000,0.000000,0.000000,0.038673,0.000000,0.000000
ZYX,0.003720,0.608146,0.066755,0.264248,0.0,0.0,0.264317,0.000000,0.000000,0.000000,...,0.012815,0.000000,0.000000,0.020495,0.000000,0.000000,0.000000,0.012873,0.000000,0.022981


In [78]:
STRING_features_completo2 = STRING_features_completo.dropna()

STRING_features_completo2

,betweenness,closeness,degree,clustering_coefficient,CNA: BLCA,CNA: BRCA,CNA: CESC,CNA: COAD,CNA: ESCA,CNA: HNSC,...,MF: KIRC,MF: KIRP,MF: LIHC,MF: LUAD,MF: LUSC,MF: PRAD,MF: READ,MF: STAD,MF: THCA,MF: UCEC
gene,,,,,,,,,,,,,,,,,,,,,
A1BG,0.002394,0.574926,0.056669,0.374335,0.0,0.0,0.167401,0.000000,0.000000,0.255263,...,0.012838,0.040970,0.050114,0.000000,0.000000,0.022774,0.028288,0.013054,0.000000,0.007712
A1CF,0.000439,0.491816,0.016191,0.216980,0.0,0.0,0.000000,0.000000,0.283951,0.000000,...,0.012698,0.000000,0.024672,0.117901,0.000000,0.090341,0.027762,0.011448,0.000000,0.010251
A2M,0.003584,0.595557,0.071267,0.274020,0.0,0.0,0.000000,0.174419,0.000000,0.000000,...,0.012836,0.040877,0.000000,0.122328,0.109769,0.022773,0.000000,0.038998,0.003601,0.019196
A2ML1,0.001648,0.451462,0.018447,0.170606,0.0,0.0,0.000000,0.184049,0.000000,0.000000,...,0.000000,0.040866,0.049805,0.040017,0.054533,0.022713,0.000000,0.063755,0.000000,0.007600
A4GALT,0.000348,0.482407,0.009821,0.179099,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.022728,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZYG11A,0.000080,0.354566,0.006370,0.380102,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.007645
ZYG11B,0.000280,0.406438,0.008626,0.265268,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.012818,0.081662,0.025013,0.000000,0.000000,0.000000,0.000000,0.038673,0.000000,0.000000
ZYX,0.003720,0.608146,0.066755,0.264248,0.0,0.0,0.264317,0.000000,0.000000,0.000000,...,0.012815,0.000000,0.000000,0.020495,0.000000,0.000000,0.000000,0.012873,0.000000,0.022981


In [79]:
STRING_features_completo2.to_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/data_final/STRING_features.tsv', sep='\t')

In [80]:
# Ficar apenas com os labels que também contém no banco de dados

label_1 = pd.merge(STRING_features_da_rede2, labels, how = 'left', on = 'gene')

label_2 = label_1.drop(columns=['betweenness', 'closeness', 'degree', 'clustering_coefficient'])

# label_2.loc[label_2['label'] == 'True', 'label'] = 'driver'
# label_2.loc[label_2['label'] == 'False', 'label'] = 'passenger'

label_3 = label_2.set_index('gene')

label_3

,label
gene,
A1BG,False
A1CF,True
A2M,False
A2ML1,False
A4GALT,False
...,...
ZYG11A,False
ZYG11B,False
ZYX,False


In [81]:
label_4 = label_3.dropna()

label_4

,label
gene,
A1BG,False
A1CF,True
A2M,False
A2ML1,False
A4GALT,False
...,...
ZYG11A,False
ZYG11B,False
ZYX,False


In [82]:
label_4.to_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/data_final/STRING_labels.tsv', sep='\t')

### PCNET

In [83]:
PCNET_network = pd.read_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/networks/PCNET_network3.tsv', sep='\t', index_col='Unnamed: 0')

PCNET_features_da_rede = pd.read_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/features_das_redes/features_pcnet.tsv', sep='\t', index_col='Unnamed: 0')

features = pd.read_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/features/unity_features.tsv', sep='\t', index_col='Unnamed: 0')

labels = pd.read_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/labels/unity_labels.tsv', sep='\t', index_col='Unnamed: 0')


C:\Users\renan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [84]:
PCNET_network.insert(len(PCNET_network.columns), 'weight', 1)

PCNET_network2 = PCNET_network.rename(columns={"gene1": "source", "gene2": "target"})

PCNET_network2

,source,target,weight
0,RNF14,UBE2Q1,1
1,UBE2Q1,UBE2Q2,1
2,TMCO1,UBE2Q1,1
3,UBAC1,UBE2Q1,1
4,UBE2Q1,WWP1,1
...,...,...,...
2719735,SLC7A10,SLC7A14,1
2719736,SLC7A11,SLC7A14,1
2719737,SELE,SELP,1
2719738,SELP,SIGLEC1,1


In [54]:
PCNET_network2.to_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/data_final/PCNET_network.tsv', sep='\t')

In [86]:
PCNET_features_da_rede2 = PCNET_features_da_rede.rename(columns={"Gene": "gene"})

# Deixando igual o número de features ao número de nós 

# A ideia é fazer um conact com as features das redes (que já são os números de nós das redes) e manter só os que deram macth com esse dataframe 'left' no caso
# Isso soluciona porque concatena e fica apenas os genes contidos na rede ppi correspondente

PCNET_features2 = pd.merge(PCNET_features_da_rede2, features, how = 'left', on = 'gene')

PCNET_features_completo = PCNET_features2.set_index('gene')

PCNET_features_completo

,betweenness,closeness,degree,clustering_coefficient,CNA: BLCA,CNA: BRCA,CNA: CESC,CNA: COAD,CNA: ESCA,CNA: HNSC,...,MF: KIRC,MF: KIRP,MF: LIHC,MF: LUAD,MF: LUSC,MF: PRAD,MF: READ,MF: STAD,MF: THCA,MF: UCEC
gene,,,,,,,,,,,,,,,,,,,,,
A1BG,0.000094,0.441093,0.014590,0.390830,0.0,0.0,0.167401,0.000000,0.000000,0.255263,...,0.012838,0.040970,0.050114,0.000000,0.000000,0.022774,0.028288,0.013054,0.000000,0.007712
A1CF,0.000931,0.500559,0.037674,0.146689,0.0,0.0,0.000000,0.000000,0.283951,0.000000,...,0.012698,0.000000,0.024672,0.117901,0.000000,0.090341,0.027762,0.011448,0.000000,0.010251
A2M,0.001133,0.529906,0.052663,0.161908,0.0,0.0,0.000000,0.174419,0.000000,0.000000,...,0.012836,0.040877,0.000000,0.122328,0.109769,0.022773,0.000000,0.038998,0.003601,0.019196
A2ML1,0.000058,0.354451,0.006895,0.265839,0.0,0.0,0.000000,0.184049,0.000000,0.000000,...,0.000000,0.040866,0.049805,0.040017,0.054533,0.022713,0.000000,0.063755,0.000000,0.007600
A3GALT2,0.000000,0.050280,0.000100,1.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013021,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZYG11A,0.000025,0.405583,0.001499,0.241667,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.007645
ZYG11B,0.000186,0.455491,0.008094,0.147546,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.012818,0.081662,0.025013,0.000000,0.000000,0.000000,0.000000,0.038673,0.000000,0.000000
ZYX,0.002006,0.578445,0.088538,0.155705,0.0,0.0,0.264317,0.000000,0.000000,0.000000,...,0.012815,0.000000,0.000000,0.020495,0.000000,0.000000,0.000000,0.012873,0.000000,0.022981


In [56]:
PCNET_features_completo.to_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/data_final/PCNET_features.tsv', sep='\t')

In [91]:
# Ficar apenas com os labels que também contém no banco de dados

label_1 = pd.merge(PCNET_features_da_rede2, labels, how = 'left', on = 'gene')

label_2 = label_1.drop(columns=['betweenness', 'closeness', 'degree', 'clustering_coefficient'])

# label_2.loc[label_2['label'] == 'True', 'label'] = 'driver'
# label_2.loc[label_2['label'] == 'False', 'label'] = 'passenger'

label_3 = label_2.set_index('gene')

label_3

,label
gene,
A1BG,False
A1CF,True
A2M,False
A2ML1,False
A3GALT2,False
...,...
ZYG11A,False
ZYG11B,False
ZYX,False


In [92]:
label_3.isnull().sum().sum()

0

In [58]:
label_3.to_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/data_final/PCNET_labels.tsv', sep='\t')

### UNITY

In [93]:
UNITY_network = pd.read_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/networks/UNITY_network3.tsv', sep='\t', index_col='Unnamed: 0')

UNITY_features_da_rede = pd.read_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/features_das_redes/features_UNITY.tsv', sep='\t', index_col='Unnamed: 0')

features = pd.read_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/features/unity_features.tsv', sep='\t', index_col='Unnamed: 0')

labels = pd.read_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/labels/unity_labels.tsv', sep='\t', index_col='Unnamed: 0')


C:\Users\renan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [94]:
UNITY_network.insert(len(UNITY_network.columns), 'weight', 1)

UNITY_network2 = UNITY_network.rename(columns={"gene1": "source", "gene2": "target"})

UNITY_network2

,source,target,weight
2,TMCO1,UBE2Q1,1
5,UBE2Q1,ZNF706,1
8,RNF115,UBE2Q1,1
10,METTL13,UBE2Q1,1
11,DHX8,UBE2Q1,1
...,...,...,...
5028704,SLC7A13,SLC7A14,1
5028705,SELE,SELP,1
5028706,SLC7A10,SLC7A13,1
5028707,SLC7A11,SLC7A13,1


In [95]:
# Criar lista de NaN para removover

UNITY_features_da_rede2 = UNITY_features_da_rede.rename(columns={"Gene": "gene"})

label_1 = pd.merge(UNITY_features_da_rede2, labels, how = 'left', on = 'gene')

label_2 = label_1.drop(columns=['betweenness', 'closeness', 'degree', 'clustering_coefficient'])

nan_values = label_2[label_2['label'].isna()]

nan_values

,gene,label
161,ACOD1,NaN
269,ADAM3B,NaN
425,AGAP6,NaN
503,AKAIN1,NaN
1070,ARMH2,NaN
...,...,...
18881,XAGE5,NaN
19545,ZNF625-ZNF20,NaN
19604,ZNF705B,NaN
19666,ZNF788P,NaN


In [96]:
# Traduzir na rede PPI os genes para NaN e remover todas as aparições

UNITY_network2.replace(nan_values.set_index('gene').to_dict()['label'], inplace = True)

UNITY_network3 = UNITY_network2.dropna()

UNITY_network3

,source,target,weight
2,TMCO1,UBE2Q1,1
5,UBE2Q1,ZNF706,1
8,RNF115,UBE2Q1,1
10,METTL13,UBE2Q1,1
11,DHX8,UBE2Q1,1
...,...,...,...
5028704,SLC7A13,SLC7A14,1
5028705,SELE,SELP,1
5028706,SLC7A10,SLC7A13,1
5028707,SLC7A11,SLC7A13,1


In [97]:
UNITY_network3.to_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/data_final/UNITY_network.tsv', sep='\t')

In [98]:
UNITY_features_da_rede2 = UNITY_features_da_rede.rename(columns={"Gene": "gene"})

# Deixando igual o número de features ao número de nós 

# A ideia é fazer um conact com as features das redes (que já são os números de nós das redes) e manter só os que deram macth com esse dataframe 'left' no caso
# Isso soluciona porque concatena e fica apenas os genes contidos na rede ppi correspondente

UNITY_features2 = pd.merge(UNITY_features_da_rede2, features, how = 'left', on = 'gene')

UNITY_features_completo = UNITY_features2.set_index('gene')

UNITY_features_completo

,betweenness,closeness,degree,clustering_coefficient,CNA: BLCA,CNA: BRCA,CNA: CESC,CNA: COAD,CNA: ESCA,CNA: HNSC,...,MF: KIRC,MF: KIRP,MF: LIHC,MF: LUAD,MF: LUSC,MF: PRAD,MF: READ,MF: STAD,MF: THCA,MF: UCEC
gene,,,,,,,,,,,,,,,,,,,,,
A1BG,0.000453,0.000012,0.042251,0.366879,0.0,0.0,0.167401,0.000000,0.000000,0.255263,...,0.012838,0.040970,0.050114,0.000000,0.000000,0.022774,0.028288,0.013054,0.000000,0.007712
A1CF,0.001153,0.000012,0.040394,0.157566,0.0,0.0,0.000000,0.000000,0.283951,0.000000,...,0.012698,0.000000,0.024672,0.117901,0.000000,0.090341,0.027762,0.011448,0.000000,0.010251
A2M,0.002013,0.000013,0.085709,0.215840,0.0,0.0,0.000000,0.174419,0.000000,0.000000,...,0.012836,0.040877,0.000000,0.122328,0.109769,0.022773,0.000000,0.038998,0.003601,0.019196
A2ML1,0.000620,0.000012,0.032686,0.360546,0.0,0.0,0.000000,0.184049,0.000000,0.000000,...,0.000000,0.040866,0.049805,0.040017,0.054533,0.022713,0.000000,0.063755,0.000000,0.007600
A3GALT2,0.000000,0.000004,0.000093,1.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013021,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZYG11A,0.000016,0.000010,0.005293,0.372656,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.007645
ZYG11B,0.000208,0.000011,0.015043,0.245172,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.012818,0.081662,0.025013,0.000000,0.000000,0.000000,0.000000,0.038673,0.000000,0.000000
ZYX,0.005280,0.000014,0.199740,0.357556,0.0,0.0,0.264317,0.000000,0.000000,0.000000,...,0.012815,0.000000,0.000000,0.020495,0.000000,0.000000,0.000000,0.012873,0.000000,0.022981


In [99]:
UNITY_features_completo2 = UNITY_features_completo.dropna()

UNITY_features_completo2

,betweenness,closeness,degree,clustering_coefficient,CNA: BLCA,CNA: BRCA,CNA: CESC,CNA: COAD,CNA: ESCA,CNA: HNSC,...,MF: KIRC,MF: KIRP,MF: LIHC,MF: LUAD,MF: LUSC,MF: PRAD,MF: READ,MF: STAD,MF: THCA,MF: UCEC
gene,,,,,,,,,,,,,,,,,,,,,
A1BG,0.000453,0.000012,0.042251,0.366879,0.0,0.0,0.167401,0.000000,0.000000,0.255263,...,0.012838,0.040970,0.050114,0.000000,0.000000,0.022774,0.028288,0.013054,0.000000,0.007712
A1CF,0.001153,0.000012,0.040394,0.157566,0.0,0.0,0.000000,0.000000,0.283951,0.000000,...,0.012698,0.000000,0.024672,0.117901,0.000000,0.090341,0.027762,0.011448,0.000000,0.010251
A2M,0.002013,0.000013,0.085709,0.215840,0.0,0.0,0.000000,0.174419,0.000000,0.000000,...,0.012836,0.040877,0.000000,0.122328,0.109769,0.022773,0.000000,0.038998,0.003601,0.019196
A2ML1,0.000620,0.000012,0.032686,0.360546,0.0,0.0,0.000000,0.184049,0.000000,0.000000,...,0.000000,0.040866,0.049805,0.040017,0.054533,0.022713,0.000000,0.063755,0.000000,0.007600
A3GALT2,0.000000,0.000004,0.000093,1.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013021,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZYG11A,0.000016,0.000010,0.005293,0.372656,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.007645
ZYG11B,0.000208,0.000011,0.015043,0.245172,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.012818,0.081662,0.025013,0.000000,0.000000,0.000000,0.000000,0.038673,0.000000,0.000000
ZYX,0.005280,0.000014,0.199740,0.357556,0.0,0.0,0.264317,0.000000,0.000000,0.000000,...,0.012815,0.000000,0.000000,0.020495,0.000000,0.000000,0.000000,0.012873,0.000000,0.022981


In [ ]:
# Vou ter que conferir todos os bancos de novo, estava concatenando e não me atentei que sempre vai bater o número com o numero de genes da rede
# Alguns genes podem não ter features, por isso preciso remover NaN de todo o banco de dados
# O dataframe de ventralidades vai ser sempre completo porque é extraido da rede mesmo, então tenho que procurar valores faltando nas features e remover a linha inteira

# Nesse caso da UNITY eu percebi porque só tenho 19.666 genes disponíveis com suas features correspondentes
# Essa rede PPI tem 19.792 genes, logo a diferença seria de 126 genes a mais que não possuem features
# Na procura por valores faltantes dá 190 isso significa que tinha valores faltantes antes (Conferir nos bancos originais do Shulte-Sasse)
# Como encontrar a diferença (Remover todas as linhas que tiver pelo menos um NaN vai afetar demais o banco?)
# O dataframe de features tá cheio de NaN (A maioria era de genes sem features mesmo)

In [100]:
UNITY_features_completo2.to_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/data_final/UNITY_features.tsv', sep='\t')

In [101]:
# Ficar apenas com os labels que também contém no banco de dados

label_1 = pd.merge(UNITY_features_da_rede2, labels, how = 'left', on = 'gene')

label_2 = label_1.drop(columns=['betweenness', 'closeness', 'degree', 'clustering_coefficient'])

# label_2.loc[label_2['label'] == 'True', 'label'] = 'driver'
# label_2.loc[label_2['label'] == 'False', 'label'] = 'passenger'

label_3 = label_2.set_index('gene')

label_3

,label
gene,
A1BG,False
A1CF,True
A2M,False
A2ML1,False
A3GALT2,False
...,...
ZYG11A,False
ZYG11B,False
ZYX,False


In [102]:
label_4 = label_3.dropna()

label_4

,label
gene,
A1BG,False
A1CF,True
A2M,False
A2ML1,False
A3GALT2,False
...,...
ZYG11A,False
ZYG11B,False
ZYX,False


In [103]:
label_4.to_csv('C:/Users/renan/Desktop/UFRGS/GNN/data/data_final/UNITY_labels.tsv', sep='\t')